Just gotta do it: need another package

https://photutils.readthedocs.io/en/stable/index.html

`conda install photutils -c astropy`

In [1]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
import astropy.coordinates as coord
from astropy import units as u
from astropy import constants as c
from astropy.io import fits
from photutils import DAOStarFinder, aperture_photometry, CircularAperture
from astropy.stats import mad_std

In [2]:
hdu = fits.open('simulated_image.fits')
# These are the actual values used, but for real data we would have to determine them from the data itself
sigma_noise = np.sqrt(5.) # variance and mean are 5
avg_bckgrd = 5.
sigma_beam = 3.
fwhm = 2.355*sigma_beam

In [3]:
# Get the true source positions and brightnesses
x_true = hdu[1].data['x_src']
y_true = hdu[1].data['y_src']
I_true = hdu[1].data['I_src']

In [4]:
# That seems to pick up on the correct background level
image = hdu[0].data
image_srcfind = image.copy()
image_med = np.median(image_srcfind) 
image_srcfind -= image_med
print(image_med)

5.000000000000681


In [5]:
plt.figure()
plt.imshow(image,vmax=50,origin='lower')
plt.title('Original data')
plt.colorbar()
plt.show()

In [6]:
bkg_sigma = mad_std(image_srcfind)
print(bkg_sigma) 
# The right answer is 
print(sigma_noise)

2.965197513650069
2.23606797749979


In [7]:
plt.imshow(image_srcfind, vmin=4*bkg_sigma,origin='lower', vmax=50)
plt.title('Threshhold image')
plt.colorbar()
plt.show()

In [22]:
daofind = DAOStarFinder(fwhm=fwhm, threshold=4.*bkg_sigma)
sources = daofind(image_srcfind) 
print(sources)

 id     xcentroid      ...        flux                mag        
--- ------------------ ... ------------------ -------------------
  1 173.81356213844984 ...  4.921071889627898  -1.730149273711321
  2  311.8237824173658 ...  6.037215102957682 -1.9520916245613469
  3 360.16889687048126 ...  7.736385200995659 -2.2213452137337693
  4 26.705071650889487 ...  5.123347077199059 -1.7738844445447277
  5  474.6480559919592 ...  5.840838252232719 -1.9161879491998903
  6  257.5426919940964 ... 4.7398506920226104  -1.689411653419401
  7  484.1200584158487 ...  7.660019934985249 -2.2105747496829538
  8  73.36782045075293 ...  8.039440354855252 -2.2630645437437424
  9  99.10155637103827 ...  5.629073035370265 -1.8760922088118657
 10 339.42613496681656 ...  4.669153554946441 -1.6730953921354894
...                ... ...                ...                 ...
 89  70.18174683474425 ...  4.856884824270572 -1.7158945119372628
 90 132.74675171543785 ...  4.443668506615263 -1.6193541338192972
 91 208.64

In [23]:
positions = [(x,y) for x,y in zip(sources['xcentroid'].data, sources['ycentroid'].data)]

In [24]:
apertures = CircularAperture(positions, r=fwhm)
#phot_table = aperture_photometry(image, apertures)

In [30]:
plt.figure(figsize=(10,8))
plt.imshow(image_srcfind, vmin=-bkg_sigma, vmax=4*bkg_sigma)
apertures.plot(color='red')
plt.colorbar(shrink=0.9)
plt.show()

In [31]:
# Let's match the sources we found against the true ones
real_indx = np.zeros_like(sources['id'])

In [33]:
for i, source in enumerate(sources):
    # Find the distance between the source which was found and the true positions
    d = np.sqrt( np.power(source['xcentroid'] - x_true, 2) + np.power(source['xcentroid'] - x_true, 2) )
    real_indx[i] = np.argmin(d)

In [47]:
plt.figure()
plt.plot(x_true, y_true, 'o')
plt.plot(sources['xcentroid'], sources['ycentroid'],'x', color='red')
plt.show()

In [53]:
plt.figure()
plt.plot(sources['xcentroid'],x_true[real_indx],'o')
plt.show()
plt.figure()
plt.hist(sources['xcentroid']-x_true[real_indx],bins=30)
plt.show()

In [54]:
plt.figure()
plt.plot(sources['ycentroid'],y_true[real_indx],'o')
plt.show()
plt.figure()
plt.hist(sources['ycentroid']-y_true[real_indx],bins=30)
plt.show()

In [55]:
plt.figure()
plt.plot(sources['peak'],I_true[real_indx],'o')
plt.plot([0,200],[0,200])
plt.show()
plt.figure()
plt.hist(sources['peak']-I_true[real_indx],bins=30)
plt.show()